# 训练一个分类器  

你已经学会如何去定义一个神经网络,计算损失值和更新网络的权重。

你现在可能在思考：数据哪里来呢？

In [2]:
%matplotlib inline

通常，当你处理图像，文本，音频和视频数据时，你可以使用标准的Python包来加载数据到一个numpy数组中.然后把这个数组转换成torch.*Tensor。

- 对于图像,有诸如Pillow,OpenCV包等非常实用
- 对于音频,有诸如scipy和librosa包
- 对于文本,可以用原始Python和Cython来加载,或者使用NLTK和SpaCy 

对于视觉,我们创建了一个torchvision包,包含常见数据集的数据加载,比如Imagenet,CIFAR10,MNIST等,和图像转换器,也就是torchvision.datasets和torch.utils.data.DataLoader。
 
这提供了巨大的便利,也避免了代码的重复。

在这个教程中,我们使用CIFAR10数据集,它有如下10个类别:`airplane`,`automobile`,`bird`,`cat`,`deer`,`dog`,`frog`,`horse`,`ship`,`truck`。

这个数据集中的图像大小为3 * 32 * 32,即,3通道,32*32像素。  
![cifar10](../assets/cifar10.png)

## 训练一个图像分类器

我们将按照下列顺序进行:

- 使用torchvision加载和归一化CIFAR10训练集和测试集.
- 定义一个卷积神经网络
- 定义损失函数
- 在训练集上训练网络
- 在测试集上测试网络

1. 加载和归一化CIFAR10  

使用torchvision加载CIFAR10是非常容易的。

In [17]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


torchvision的输出是[0,1]的PILImage图像,我们把它转换为归一化范围为[-1, 1]的张量。

注意
> 如果在Windows上运行时出现BrokenPipeError，尝试将torch.utils.data.DataLoader()的num_worker设置为0。

In [20]:
transform = transforms.Compose([transforms.ToTensor(), 
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='../dataset/', train=True,
                                       download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
#                                           shuffle=True, num_workers=2)
trainloader = DataLoader(trainset, batch_size=batch_size,
                        shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../dataset/', train=False,
                                      download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, 
                                        shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


AttributeError: 'Compose' object has no attribute 'Compose'